In [15]:
import torch
import module_model
import lightning as L
import pennylane as qml
from pennylane import numpy as np
L.seed_everything(0)

Global seed set to 0


0

In [16]:
def encoding(num_ir_qubits, num_nr_qubits, mode):
    # rotation encoding on pennylane simulator
    def pennylane_encoding(_input, control_values):
        for i in range(num_nr_qubits):
            ctrl_H = qml.ctrl(qml.Hadamard, control=range(num_ir_qubits), control_values=control_values)
            ctrl_H(wires=num_ir_qubits+i)
            ctrl_R = qml.ctrl(qml.Rot, control=range(num_ir_qubits), control_values=control_values)
            ctrl_R(theta=_input[0], phi=_input[1], omega=_input[2], wires=num_ir_qubits+i)

    # rotation encoding on qiskit
    num_wk_qubits = num_ir_qubits - 1
    def qiskit_encoding(_input, control_values):
        theta, phi, omega = _input[0], _input[1], _input[2]
        # see N.C. page.184
        for i in range(num_nr_qubits):
            # control values
            for q in range(num_ir_qubits):
                if control_values[q] == 0:
                    qml.PauliX(wires=q)
            # toffoli transformation
            if num_ir_qubits >= 2:
                qml.Toffoli(wires=(0, 1, num_ir_qubits))
            for q in range(num_wk_qubits-1):
                qml.Toffoli(wires=(2+q, num_ir_qubits+q, num_ir_qubits+1+q))
            # ctrl_H: decomposed by H = i Rx(pi) Ry(pi/2) (if complete graph with power of 2 nodes -> relative phase i becomes global)
            target_qubit = num_ir_qubits + num_wk_qubits + i
            qml.CRY(np.pi/2, wires=(num_ir_qubits + num_wk_qubits - 1, target_qubit))
            qml.CRX(np.pi, wires=(num_ir_qubits + num_wk_qubits - 1, target_qubit))
            # ctrl_R: Rot(phi, theta, omega) = Rz(omega) Ry(theta) Rz(phi)
            qml.CRZ(phi, wires=(num_ir_qubits + num_wk_qubits - 1, target_qubit))
            qml.CRY(theta, wires=(num_ir_qubits + num_wk_qubits - 1, target_qubit))
            qml.CRZ(omega, wires=(num_ir_qubits + num_wk_qubits - 1, target_qubit))
            # toffoli inverse transformation
            for q in reversed(range(num_wk_qubits-1)):
                qml.Toffoli(wires=(2+q, num_ir_qubits+q, num_ir_qubits+1+q))
            if num_ir_qubits >= 2:
                qml.Toffoli(wires=(0, 1, num_ir_qubits))
            # control values
            for q in range(num_ir_qubits):
                if control_values[q] == 0:
                    qml.PauliX(wires=q)

    if mode == "pennylane":
        return pennylane_encoding
    elif mode == "qiskit":
        return qiskit_encoding

In [18]:
IR = 3 # number of qubits in IR
NR = 2 # number of qubits in NR

for i in range(10):
    L.seed_everything(i) # to make the random initialized weights reproducible
    pennylane_circuit = module_model.QCGNN(IR, NR, 1, 1, ctrl_enc=encoding(IR, NR, "pennylane"))

    L.seed_everything(i) # to make the random initialized weights reproducible
    qiskit_circuit    = module_model.QCGNN(IR, NR, 1, 1, ctrl_enc=encoding(IR, NR, "qiskit"))

    N = np.random.randint(2**IR) + 1
    x = torch.rand(N, 3)

    y_pennylane = pennylane_circuit(x).reshape(-1)
    y_qiskit    = qiskit_circuit(x).reshape(-1)
    print(f"Pennylane:\n - {y_pennylane}")
    print(f"Qiskit:\n - {y_qiskit}")
    print(f"Difference norm = {torch.norm(y_pennylane-y_qiskit, p=2):.10f}")
    print(f"\n{'-'*100}\n")

Global seed set to 0
Global seed set to 0
Global seed set to 1
Global seed set to 1


# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 2, 2)
Pennylane:
 - tensor([ 8.6146, -8.7186,  5.7605, -5.1836,  6.5973, -5.9904,  4.3638, -4.1662,
         4.3642, -4.3749], grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([ 8.6146, -8.7186,  5.7605, -5.1836,  6.5973, -5.9904,  4.3638, -4.1662,
         4.3642, -4.3749], grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000011680

----------------------------------------------------------------------------------------------------

# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 2, 2)


Global seed set to 2
Global seed set to 2
Global seed set to 3
Global seed set to 3
Global seed set to 4
Global seed set to 4


Pennylane:
 - tensor([ 20.5682, -20.7537,  23.8379, -23.8621,  31.6828, -31.7217,  25.8966,
        -26.0866,  27.8525, -28.1735,  28.6361, -28.8018],
       grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([ 20.5682, -20.7537,  23.8379, -23.8621,  31.6828, -31.7217,  25.8966,
        -26.0866,  27.8525, -28.1735,  28.6361, -28.8018],
       grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000071366

----------------------------------------------------------------------------------------------------

# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 2, 2)
Pennylane:
 - tensor([-15.7156,  -7.3684], grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([-15.7156,  -7.3684], grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000000000

----------------------------------------------------------------------------------------------------

# ModelLog: Quantum device  = default.qubit | Qu

Global seed set to 5
Global seed set to 5
Global seed set to 6
Global seed set to 6


Pennylane:
 - tensor([-41.5523,  18.4090, -45.9006,  23.4644, -39.4279,  13.9275],
       grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([-41.5523,  18.4090, -45.9006,  23.4644, -39.4279,  13.9275],
       grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000038147

----------------------------------------------------------------------------------------------------

# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 2, 2)
Pennylane:
 - tensor([-41.5441,  -6.3623, -49.1275,  -6.6868, -41.9647,  -7.5990, -42.8934,
         -6.0298], grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([-41.5441,  -6.3623, -49.1275,  -6.6868, -41.9647,  -7.5990, -42.8934,
         -6.0298], grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000018468

----------------------------------------------------------------------------------------------------

# ModelLog: Quantum device  = default.qubit | Qu

Global seed set to 7
Global seed set to 7


Pennylane:
 - tensor([-2.7514,  1.0339, -3.1319,  0.5160, -6.0548, -2.8743],
       grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([-2.7514,  1.0339, -3.1319,  0.5160, -6.0548, -2.8743],
       grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000006848

----------------------------------------------------------------------------------------------------

# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 2, 2)


Global seed set to 8
Global seed set to 8
Global seed set to 9
Global seed set to 9


Pennylane:
 - tensor([-12.8231,   0.9222, -17.1042,   1.0457, -16.3320,   4.0627, -17.8126,
          2.9222, -16.4908,   3.7717, -16.4782,   4.3244, -16.4099,   4.5318,
        -13.7260,   2.2017], grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([-12.8231,   0.9222, -17.1042,   1.0457, -16.3320,   4.0627, -17.8126,
          2.9222, -16.4908,   3.7717, -16.4782,   4.3244, -16.4099,   4.5318,
        -13.7260,   2.2017], grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000016082

----------------------------------------------------------------------------------------------------

# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 2, 2)
Pennylane:
 - tensor([13.9520, 10.9789, 12.5315, 10.4392, 14.4646, 10.2833, 13.6068,  6.8219],
       grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([13.9520, 10.9789, 12.5315, 10.4392, 14.4646, 10.2833, 13.6068,  6.8219],
       grad_fn=<Reshap